Перед вами задача классификации космических объектов.

Целевая переменная называется class.

Решите задачу методами случайного леса, и рассмотренными на уроке бустингами.

Оцените результаты с помощью метрик presicion и recall.

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error, accuracy_score
import math

In [2]:
df = pd.read_csv('star_classification.csv')

In [3]:
df.head()

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   obj_ID       100000 non-null  float64
 1   alpha        100000 non-null  float64
 2   delta        100000 non-null  float64
 3   u            100000 non-null  float64
 4   g            100000 non-null  float64
 5   r            100000 non-null  float64
 6   i            100000 non-null  float64
 7   z            100000 non-null  float64
 8   run_ID       100000 non-null  int64  
 9   rerun_ID     100000 non-null  int64  
 10  cam_col      100000 non-null  int64  
 11  field_ID     100000 non-null  int64  
 12  spec_obj_ID  100000 non-null  float64
 13  class        100000 non-null  object 
 14  redshift     100000 non-null  float64
 15  plate        100000 non-null  int64  
 16  MJD          100000 non-null  int64  
 17  fiber_ID     100000 non-null  int64  
dtypes: float64(10), int64(7),

In [5]:
df['class'].unique()

array(['GALAXY', 'QSO', 'STAR'], dtype=object)

In [6]:
X = df.drop(['class'], axis = 1)

In [7]:
X.head()

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,0.116123,6121,56187,842


In [8]:
Y = df['class']

In [9]:
Y.head()

0    GALAXY
1    GALAXY
2    GALAXY
3    GALAXY
4    GALAXY
Name: class, dtype: object

## случайный лес

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 0)

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, Y_train)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1,
                       random_state=42)

In [14]:
pred_rnd = rnd_clf.predict(X_test)
accuracy_score(Y_test, pred_rnd)

0.96064

In [15]:
from sklearn.metrics import confusion_matrix 

In [16]:
cm = confusion_matrix(Y_test, pred_rnd)
cm

array([[14405,   266,   134],
       [  582,  4160,     1],
       [    1,     0,  5451]], dtype=int64)

In [17]:
precision_score(Y_test, pred_rnd, average='macro')

0.9589450802605793

In [18]:
recall_score(Y_test, pred_rnd, average='macro')

0.9499602324382606

## бустинг

In [19]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [20]:
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 4, random_state=42),
    n_estimators=100,
    algorithm="SAMME",
    learning_rate=0.5,
    random_state=42)

In [21]:
ada_clf.fit(X_train, Y_train)

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(max_depth=4,
                                                         random_state=42),
                   learning_rate=0.5, n_estimators=100, random_state=42)

In [22]:
pred_ada = ada_clf.predict(X_test)
accuracy_score(Y_test, pred_ada)

0.97576

In [23]:
precision_score(Y_test, pred_ada, average='macro')

0.9730769631378019

In [24]:
recall_score(Y_test, pred_ada, average='macro')

0.9706760649020927

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

In [26]:
gbc = GradientBoostingClassifier(n_estimators = 100,
                                max_depth = 4,
                                random_state = 15)
gbc.fit(X_train,Y_train)

GradientBoostingClassifier(max_depth=4, random_state=15)

In [27]:
pred_gbc = gbc.predict(X_test)
accuracy_score(Y_test, pred_gbc)

0.97744

In [28]:
precision_score(Y_test, pred_gbc, average='macro')

0.9773965890772988

In [29]:
recall_score(Y_test, pred_gbc, average='macro')

0.9700363369944784